In [ ]:
import sys as _sys
import os

current_path = os.path.abspath(os.getcwd())

split = current_path.split("inverse_geometric_locomotion")
if len(split)<2:
    print("Please rename the repository 'inverse_geometric_locomotion'")
    raise ValueError
path_to_python_scripts = os.path.join(split[0], "inverse_geometric_locomotion/python/")
path_to_notifications = os.path.join(split[0], "inverse_geometric_locomotion/notebooks/notifications/")
path_to_settings = os.path.join(split[0], "inverse_geometric_locomotion/python/figures/")
path_to_cubic_splines = os.path.join(split[0], "inverse_geometric_locomotion/ext/torchcubicspline/")
path_to_output = os.path.join(split[0], "inverse_geometric_locomotion/output/")
path_to_data = path_to_output

_sys.path.insert(0, path_to_python_scripts)
_sys.path.insert(0, path_to_settings)
_sys.path.insert(0, path_to_cubic_splines)

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import torch
from vis_utils import print_json_data

## Load data

This notebook can be used to read files generated from executing `python/experiments/astronaut.py`. The following list assumes you have executed the experiment script successfully with the flag `--trial_number=0`. 

As you run more experiments with different `trial_number`, you can add the output path to the list `exp_file_names` with the corresponding file locations. The naming convention for `.json` files is `*_opt_{trial_number}.json`, where `{trial_number}` always has two digits.

In [ ]:
exp_file_names = [
    "astronaut_all_vertices/astronaut_opt_00.json",
]

list_js_loads = []
for exp_file_name in exp_file_names:
    with open(os.path.join(path_to_data, exp_file_name)) as jsonFile:
        js_load = json.load(jsonFile)

    print(exp_file_name)
    print_json_data(js_load)
    list_js_loads.append(js_load)

print("\nLoaded {} experiments.".format(len(list_js_loads)))

## Plot edge length preservation

Make sure the soft isometry constraint is small enough by plotting its evolution. 

You can select which experiment to plot from the list in the previous cell `exp_file_names` by changing `exp_id` accordingly. By default, the first element of the list is used.

In [ ]:
exp_id = 0 # You can change this to plot a different experiment
js_load = list_js_loads[exp_id]

pos_ = torch.tensor(js_load['pos'])
pos = np.array(js_load['pos'])
edges_graph = torch.tensor(js_load['optimization_settings']['edges_graph'])
length_threshold = js_load['optimization_settings']['repulsion_length_threshold']
n_edge_disc = js_load['optimization_settings']['n_edge_disc']
n_ts = pos.shape[0]
g = np.array(js_load['g'])
gt = np.array(js_load['optimization_settings']['gt'])
params_opt = np.array(js_load['optimization_settings']['params_opt'])
initial_edge_lengths = torch.tensor(js_load['optimization_settings']['initial_edge_lengths'])

new_edge_lengths = torch.linalg.norm((pos_[:, edges_graph[:, 0]] - pos_[:, edges_graph[:, 1]]), dim=-1) # shape (n_steps, n_edges)

n_edges = edges_graph.shape[0]
alphas = torch.linspace(0.0, 1.0, n_edge_disc + 2)[1:-1].reshape(1, 1, -1, 1) # shape (n_edge_disc,)
edge_disc = pos_[:, edges_graph[:, 0]].unsqueeze(2) * (1.0 - alphas) + pos_[:, edges_graph[:, 1]].unsqueeze(2) * alphas # shape (n_steps, n_edges, n_edge_disc, 3)
edge_disc_pd = torch.sqrt(1.0e-12 + torch.sum((edge_disc.reshape(pos_.shape[0], n_edges, 1, n_edge_disc, 1, 3) - edge_disc.reshape(pos_.shape[0], 1, n_edges, 1, n_edge_disc, 3)) ** 2, dim=-1)) # shape (n_steps, n_edges, n_edges, n_edge_disc, n_edge_disc)
idx_upper_edge_disc = torch.triu_indices(n_edge_disc, n_edge_disc, offset=0)
edge_to_edge_min_dist = torch.min(edge_disc_pd[:, :, :, idx_upper_edge_disc[0], idx_upper_edge_disc[1]], dim=-1)[0] # shape (n_steps, n_edges, n_edges)
idx_upper_edges = torch.triu_indices(n_edges, n_edges, offset=1)
pairwise_distances_edges = edge_to_edge_min_dist[:, idx_upper_edges[0], idx_upper_edges[1]] # shape (n_steps, n_edges * (n_edges-1) / 2)
min_pd_diff = torch.min(pairwise_distances_edges- length_threshold, dim=-1)[0] # shape (n_steps,)
energy_edges_per_time = 0.5 * torch.mean(torch.relu(- (pairwise_distances_edges - length_threshold) ** 2 * torch.log(pairwise_distances_edges / length_threshold)), dim=-1) # IPC

plt.figure(figsize=(6, 3))
plt.plot(100.0 * torch.mean(torch.abs(new_edge_lengths - initial_edge_lengths.reshape(1, -1)), dim=0) / initial_edge_lengths, lw=3.0)
plt.title("Edge length change (%)")
plt.show()

plt.figure(figsize=(6, 3))
plt.plot(min_pd_diff, lw=3.0)
plt.title("Minimum edge to edge distance - threshold")
plt.show()